# Data Exporting

In [ ]:
!pwd

In [ ]:
!pip uninstall cytokinin -y

In [ ]:
!pip install ./../cytokinin

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from pathlib import Path

In [ ]:
import logging
logging.basicConfig()
log = logging.getLogger()
log.setLevel(logging.INFO)

In [ ]:
import cytokinin as ck
from cytokinin.data import take_data

In [ ]:
root = Path('./../') # quite the same as str

In [ ]:
# Set an example dir for images files
MOCKS = root.joinpath('./cytokinin/cytokinin/tests/mocks/')
IMGS = MOCKS/'imgs' # this is another Path object
os.listdir(str(IMGS))

## Expell

In [ ]:
c = take_data('images').store_filesnames_from_folder(IMGS/'dog')
c.filesnames.head()

In [ ]:
# BEFORE TO RUN THIS: create this folder (mocks/bin) yourself in order to see what happen
path = str(MOCKS/'bin')
inthisnewfolder = 'new_images'
namefilesas = None
c.expell_to(path, inthisnewfolder, namefilesas)

expell_to: namefilesas 'data'

In [ ]:
path = str(MOCKS/'bin')
inthisnewfolder = 'new_images2'
namefilesas = 'data'
c.expell_to(path, inthisnewfolder, namefilesas)

expell_to: namefilesas 'folder'

In [ ]:
path = str(MOCKS/'bin')
inthisnewfolder = 'new_images3'
namefilesas = 'folder'
c.expell_to(path, inthisnewfolder, namefilesas)

## Generate a ml df

In [ ]:
# Create a Data from two roots: dogs and stones
dogs = take_data('images').store_filesnames_from_folder(IMGS.joinpath('dog'))
stones = take_data('images').store_filesnames_from_folder(IMGS.joinpath('stone'))
dands = dogs.copy().add_from_data(stones)

csv_url = MOCKS/'labels'/'dogsandstones_labes.csv'
dands.label_from_csv(csv_url, col='Y')
print(dands)

#### with labels as category names

In [ ]:
dands.filesnames_to_ml_df()

#### with labels as category numbers

In [ ]:
dands.filesnames_to_ml_df(y_as='classnum').copy()

#### with labels as 1-hot encoded

In [ ]:
dands.filesnames_to_ml_df(y_as='1hot').copy()

## Export

### to Keras ImageDataGenerator

Let's try to export a not labeled data, equal a single class data, beacuse cytokinin assumes that if you're sure to ask not labeled data it means they are all from a single class.

In [ ]:
# Create a Data from two roots: dogs and stones
dogs = take_data('images').store_filesnames_from_folder(IMGS.joinpath('dog'))
stones = take_data('images').store_filesnames_from_folder(IMGS.joinpath('stone'))
dands = dogs.copy().add_from_data(stones)
print(dands)

In [ ]:
dands.export_to_keras()

Now use well labeled files (2 classes)

In [ ]:
# Create a Data from two roots: dogs and stones
dogs = take_data('images').store_filesnames_from_folder(IMGS.joinpath('dog'))
stones = take_data('images').store_filesnames_from_folder(IMGS.joinpath('stone'))
dands = dogs.copy().add_from_data(stones)

csv_url = MOCKS/'labels'/'dogsandstones_labes.csv'
dands.label_from_csv(csv_url, col='Y')
print(dands)

In [ ]:
idg = dands.export_to_keras()
idg

it also accepts arguments

In [ ]:
idg_args = {
    'rescale': 1./255,
    'shear_range': 0.2,
    'zoom_range': 0.2,
    'horizontal_flip': True
}
ke_gen = dands.export_to_keras(imagedatagenerator_args=idg_args)

##### Fit a Keras model

In [ ]:
### This part is for preparing a Keras model ####

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD, Adam

n_classes = 2

### Model definition ###
model = Sequential()
model.add( Dense(100, activation='relu', input_shape=(256,256,3)) )
model.add( Dense(60, activation='relu') )
model.add( Dense(10, activation='relu') )
model.add( Flatten() )
model.add( Dense(n_classes, activation='softmax') )

### Compile ###
sgd = SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#imsize = (64,64)


....and fit!

In [ ]:
### Fit ###
model.fit_generator(
        idg,
        #steps_per_epoch=20,
        epochs=3,
        #validation_data=validation_generator,
        validation_steps=1)

#### to FastAI

In [ ]:
# Create a Data from two roots: dogs and stones
dogs = take_data('images').store_filesnames_from_folder(IMGS.joinpath('dog'))
stones = take_data('images').store_filesnames_from_folder(IMGS.joinpath('stone'))
dands = dogs.copy().add_from_data(stones)

csv_url = MOCKS/'labels'/'dogsandstones_labes.csv'
dands.label_from_csv(csv_url, col='Y')
print(dands)

In [ ]:
from fastai.vision import ImageDataBunch,ImageList
from fastai.vision.transform import get_transforms
tfms_arg = get_transforms() # https://docs.fast.ai/vision.transform.html
idb_args = {
    'ds_tfms': tfms_arg,
    'valid_pct': 0,
    'bs': 40,
#     'label_col': None, #label column
}

fastai_idb = dands.export_to_fastAI(imagedatabunch_args=idb_args)
fastai_idb

In [ ]:
type(fastai_idb)

In [ ]:
fastai_idb.open( fastai_idb.items[1] )

nice doggo

#### to Pytorch

... coming soon